In [ ]:
import os
import sys

In [ ]:
sys.path.insert(0, os.path.abspath(".."))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input

from magic_packet import datasets, features
from magic_packet.models.resnet8 import resnet8

In [ ]:
# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Dataset

In [ ]:
(train_ds, val_ds, test_ds), ds_info = datasets.load(
    "mini_speech_commands",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    shuffle_files=True,
    with_info=True,
)
names = ds_info.features["label"].names

# Build and train model

## Preprocess datasets

In [ ]:
def get_mfcc_and_label(example):
    audio, label = example["audio"], example["label"]
    normalized = features.normalize(audio)
    # Add a `channels` dimension, so that the spectrogram can be used
    # as image-like input data with convolution layers (which expect
    # shape (`batch_size`, `height`, `width`, `channels`).
    mfcc = features.mfcc(normalized)[..., tf.newaxis]
    return mfcc, label


def preprocess_dataset(ds):
    return ds.map(map_func=get_mfcc_and_label, num_parallel_calls=tf.data.AUTOTUNE)


train_ds = preprocess_dataset(train_ds)
val_ds = preprocess_dataset(val_ds)
test_ds = preprocess_dataset(test_ds)

## Model

In [ ]:
for mfcc, _ in train_ds.take(1):
    input_shape = mfcc.shape
print("Input shape:", input_shape)

n_labels = len(names)
inputs = Input(shape=input_shape)
model = resnet8(inputs, n_labels)
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

## Fit

In [ ]:
batch_size = 64
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

In [ ]:
train_ds = train_ds.cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics["loss"], metrics["val_loss"])
plt.legend(["loss", "val_loss"])
plt.show()

## Evaluate

In [ ]:
test_audio = []
test_labels = []

for audio, label in test_ds:
    test_audio.append(audio.numpy())
    test_labels.append(label.numpy())

test_audio = np.array(test_audio)
test_labels = np.array(test_labels)

In [ ]:
y_pred = np.argmax(model.predict(test_audio), axis=1)
y_true = test_labels

test_acc = sum(y_pred == y_true) / len(y_true)
print(f"Test set accuracy: {test_acc:.0%}")

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, xticklabels=names, yticklabels=names, annot=True, fmt="g")
plt.xlabel("Prediction")
plt.ylabel("Label")
plt.show()

In [ ]:
for audio, label in test_ds.take(1):
    prediction = model(tf.expand_dims(audio, axis=0))
    plt.bar(names, tf.nn.softmax(prediction[0]))
    plt.title(f'Predictions for "{names[label]}"')
    plt.show()